# Single Responsibility

In [68]:
from abc import ABCMeta, abstractmethod

In [69]:
#before
class bank_account:
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def get_balance(): pass
    
    @abstractmethod
    def deposit(amount): pass
        
    @abstractmethod
    def withdraw(amount): pass
    
    @abstractmethod
    def add_interest(amount): pass

We can have a checking account or a savings account
Most checking accounts do not add interest, ever!
So we seperate out this respsonsibility to only the savings_account

In [70]:
#after
class bank_account:
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def get_balance(): pass
    
    @abstractmethod
    def deposit(amount): pass
        
    @abstractmethod
    def withdraw(amount): pass
    
class checking_account(bank_account): pass
    
class savings_account(bank_account):    

    def add_interest(amount): 
        return get_balance() + amount

Another example: 

In [71]:
#before
class fancy_music_player:
    def __init__(): pass
    
    def change_folders(): pass
    
    def increase_volume(): pass 
    
    def decrease_volume(): pass

This music player is responsible not only for switching between folders, but also volume control. i.e. more than one responsiblity

In [72]:
#after
class volume():
    def change(): pass

class folder():
    def change(): pass
    
class fancy_music_player:
    volume_control = volume()
    folder = folder()

    def some_method(): 
        volume_control.change()
        folder.change()
    

# Open and close

In [73]:
#example from https://lostechies.com/joeocampo/2008/03/21/ptom-the-open-closed-principle/
#before - step 1 filter by color

class product_filter:
    def by_color(self, products, productColor):
        for product in products:
            if product.Color == productColor:
                yield product;

In [74]:
#before - step 2 filter by size and color
class product_filter:
    def by_color(self, products, product_color):
        for product in products:
            if product.color == product_color:
                yield product;
                
    def by_size(self, products, product_size):
        for product in products:
            if product.size == product_size:
                yield product;
                
    def by_size_and_color(self, products, product_size, product_color):
        for product in products:
            if product.size == product_size and product.color == product_color:
                yield product;
                
#called 
products = []#array of objects
pf = product_filter()
pf.by_color(products, 'green')
pf.by_size(products, 10)
pf.by_size_and_color(products, 10, 'green')

<generator object by_size_and_color at 0x1067279d8>

What if one more filterable attribute is requested
- we have no choice but to modify above class? YES === **open** to modification
- can we extend above class? NO === **closed** for extension


In [75]:
#after
class product_filter:
    def by(self, products, product_filter_specification):
            return product_filter_specification.filter(products)

class product_filter_specification:
    def filter(self, products):
        return self.apply_filter(products)

    @abstractmethod 
    def apply_filter(self, products): pass
    

class color_filter_specification(product_filter_specification):

    def color_filter_specification(self, product_color):
        self.product_color = product_color

    def apply_filter(self, products):
        for product in products:
            if product.color == product_color:
                yield product;
                
class size_filter_specification(product_filter_specification):

    def size_filter_specification(self, product_size):
        self.product_size = product_size

    def apply_filter(self, products):
        for product in products:
            if product.size == product_size:
                yield product;
                
class size_and_color_filter_specification(product_filter_specification):

    def __init__(self, product_size, product_color):
        self.product_size = product_size
        self.product_color = product_color

    def apply_filter(self, products):
        for product in products:
            if product.size == product_size and product.color == product_color:
                yield product;
                
#called 
products = [] #array of objects
filter_spec = size_and_color_filter_specification(10, 'green')
pf = product_filter()
pf.by(products, filter_spec)


<generator object apply_filter at 0x1072187e0>

Now when a new filter critera comes up, all you need to do is extend the specification, and add a new class without modifying existing classes.

# Liskov substitution



In [76]:
class rectangle:
    def set_width(self, width): 
        self.width = width
    def set_height(self, height): 
        self.height = height
    def get_area(self): 
        return self.width * self.height
    
class square(rectangle):
    def set_width(self, width): 
        self.width = width
        self.height = width
    def set_height(self, height): 
        self.width = height
        self.height = height
    
s = square() #assume some factory returns this object
s.set_width(10)
s.set_height(34)
s.get_area()


1156

# Interface segregation


In [77]:
#example from https://lostechies.com/rayhouston/2008/03/15/ptom-the-interface-segregation-principle/
#before 1
class animal:
    def feed(): pass
class dog(animal):
    def feed(): pass
class snake(animal):
    def feed(): pass
    

In [78]:
#before 2
class animal:
    def feed(): pass
    def groom(): pass
class dog(animal):
    def feed(): pass
    def groom(): pass #OK
class snake(animal):
    def feed(): pass
    def groom(): pass #yikes!!

In [79]:
#after
class pet:
    def groom(): pass
class animal:
    def feed(): pass
class dog(animal, pet):
    def feed(): pass
    def groom(): pass #OK
class snake(animal):
    def feed(): pass
     #yay no need to groom!

# Dependency inversion

In [80]:
# example from http://code.tutsplus.com/tutorials/solid-part-4-the-dependency-inversion-principle--net-36872
#before depending on concrete classes
class ebook_reader():
    def __init__(self, pdf_book): 
        self.pdf_book = pdf_book
        
    def read(self):
        self.pdf_book.read()
        
class pdf_book:
    def read(): pass

In [81]:
#after 1
class e_book():
    def read(): pass
    
class e_book_reader():
    def __init__(self, e_book): 
        self.e_book = e_book
        
    def read(self):
        self.ebook.read()
        
class pdf_book(e_book):
    def read(): pass

In [82]:
#after 2
class e_book():
    def read(): pass
    
class e_book_reader():
    def __init__(self, e_book): 
        self.e_book = e_book
        
    def read(self):
        self.ebook.read()
        
class pdf_book(e_book):
    def read(): pass
    
class doc_book(e_book):
    def read(): pass